In [ ]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
T=10
all=np.load(f'Data/dataset_ld_{ld_dim}_{9}.npy')
all=all[:200]
for i in range(9):
    x=np.load(f'Data/dataset_ld_{ld_dim}_{i}.npy')
    all=np.concatenate((x[:200],all))

mnist_images=all
np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,256,448,496,510])
min_array=np.array([0.05,0.01,0.005])
layer_array=np.array([5,10,20,50]) #5,10,20,50
print(NUM_QUBITS)
print(T)
zero = torch.zeros(BATCH_SIZE, 2**NUM_QUBITS-ld_dim).to(device)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    

                    # concatenate the two tensors along the second dimension
                    input_batch = torch.cat((input_batch, zero), dim=1)
                    target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer} QC{qc}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'ancilla_{Q_ANCILLA}/all_thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'ancilla_{Q_ANCILLA}/all_loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

9
10
(11,)
0
T=10 Epoch: 1/40 - Loss: 0.8773 b_loss=0.8773 - T: 34.03s/epoch ,tempo_previto=1360.76 min0.05 nl5 QC0
1
T=10 Epoch: 2/40 - Loss: 0.7029 b_loss=0.7029 - T: 33.89s/epoch ,tempo_previto=1354.51 min0.05 nl5 QC0
2
T=10 Epoch: 3/40 - Loss: 0.5912 b_loss=0.5912 - T: 34.08s/epoch ,tempo_previto=1361.58 min0.05 nl5 QC0
3
T=10 Epoch: 4/40 - Loss: 0.5375 b_loss=0.5375 - T: 32.96s/epoch ,tempo_previto=1316.04 min0.05 nl5 QC0
4
T=10 Epoch: 5/40 - Loss: 0.4799 b_loss=0.4799 - T: 32.80s/epoch ,tempo_previto=1309.31 min0.05 nl5 QC0
5
T=10 Epoch: 6/40 - Loss: 0.4777 b_loss=0.4777 - T: 32.71s/epoch ,tempo_previto=1305.14 min0.05 nl5 QC0
6
T=10 Epoch: 7/40 - Loss: 0.4750 b_loss=0.4750 - T: 32.99s/epoch ,tempo_previto=1315.61 min0.05 nl5 QC0
7
T=10 Epoch: 8/40 - Loss: 0.4656 b_loss=0.4656 - T: 32.55s/epoch ,tempo_previto=1297.72 min0.05 nl5 QC0
8
T=10 Epoch: 9/40 - Loss: 0.4604 b_loss=0.4604 - T: 32.74s/epoch ,tempo_previto=1304.86 min0.05 nl5 QC0
9
T=10 Epoch: 10/40 - Loss: 0.4417 b_loss=0.

T=10 Epoch: 38/40 - Loss: 0.2493 b_loss=0.2350 - T: 32.73s/epoch ,tempo_previto=1179.31 min0.01 nl5 QC0
38
T=10 Epoch: 39/40 - Loss: 0.2446 b_loss=0.2350 - T: 32.77s/epoch ,tempo_previto=1180.15 min0.01 nl5 QC0
39
T=10 Epoch: 40/40 - Loss: 0.2450 b_loss=0.2350 - T: 32.95s/epoch ,tempo_previto=1186.05 min0.01 nl5 QC0
(11,)
0
T=10 Epoch: 1/40 - Loss: 0.7870 b_loss=0.7870 - T: 32.78s/epoch ,tempo_previto=1092.19 min0.005 nl5 QC0
1
T=10 Epoch: 2/40 - Loss: 0.5468 b_loss=0.5468 - T: 33.10s/epoch ,tempo_previto=1102.36 min0.005 nl5 QC0
2
T=10 Epoch: 3/40 - Loss: 0.4178 b_loss=0.4178 - T: 33.05s/epoch ,tempo_previto=1100.01 min0.005 nl5 QC0
3
T=10 Epoch: 4/40 - Loss: 0.3566 b_loss=0.3566 - T: 32.86s/epoch ,tempo_previto=1093.31 min0.005 nl5 QC0
4
T=10 Epoch: 5/40 - Loss: 0.3226 b_loss=0.3226 - T: 32.92s/epoch ,tempo_previto=1094.74 min0.005 nl5 QC0
5
T=10 Epoch: 6/40 - Loss: 0.3093 b_loss=0.3093 - T: 32.93s/epoch ,tempo_previto=1094.51 min0.005 nl5 QC0
6
T=10 Epoch: 7/40 - Loss: 0.2907 b_loss

T=10 Epoch: 34/40 - Loss: 0.4125 b_loss=0.4078 - T: 32.76s/epoch ,tempo_previto=1269.86 min0.05 nl5 QC256
34
T=10 Epoch: 35/40 - Loss: 0.4380 b_loss=0.4078 - T: 32.86s/epoch ,tempo_previto=1273.19 min0.05 nl5 QC256
35
T=10 Epoch: 36/40 - Loss: 0.3970 b_loss=0.3970 - T: 32.86s/epoch ,tempo_previto=1272.82 min0.05 nl5 QC256
36
T=10 Epoch: 37/40 - Loss: 0.4314 b_loss=0.3970 - T: 34.18s/epoch ,tempo_previto=1323.44 min0.05 nl5 QC256
37
T=10 Epoch: 38/40 - Loss: 0.4495 b_loss=0.3970 - T: 33.02s/epoch ,tempo_previto=1278.00 min0.05 nl5 QC256
38
T=10 Epoch: 39/40 - Loss: 0.4238 b_loss=0.3970 - T: 33.29s/epoch ,tempo_previto=1287.94 min0.05 nl5 QC256
39
T=10 Epoch: 40/40 - Loss: 0.4417 b_loss=0.3970 - T: 32.91s/epoch ,tempo_previto=1272.44 min0.05 nl5 QC256
(11,)
0
T=10 Epoch: 1/40 - Loss: 0.7806 b_loss=0.7806 - T: 32.94s/epoch ,tempo_previto=1185.15 min0.01 nl5 QC256
1
T=10 Epoch: 2/40 - Loss: 0.5903 b_loss=0.5903 - T: 33.03s/epoch ,tempo_previto=1188.11 min0.01 nl5 QC256
2
T=10 Epoch: 3/40 -

T=10 Epoch: 30/40 - Loss: 0.2287 b_loss=0.2287 - T: 33.12s/epoch ,tempo_previto=1065.26 min0.005 nl5 QC256
30
T=10 Epoch: 31/40 - Loss: 0.2312 b_loss=0.2287 - T: 33.21s/epoch ,tempo_previto=1067.55 min0.005 nl5 QC256
31
T=10 Epoch: 32/40 - Loss: 0.2345 b_loss=0.2287 - T: 33.23s/epoch ,tempo_previto=1067.70 min0.005 nl5 QC256
32
T=10 Epoch: 33/40 - Loss: 0.2412 b_loss=0.2287 - T: 33.04s/epoch ,tempo_previto=1061.15 min0.005 nl5 QC256
33
T=10 Epoch: 34/40 - Loss: 0.2437 b_loss=0.2287 - T: 32.89s/epoch ,tempo_previto=1055.85 min0.005 nl5 QC256
34
T=10 Epoch: 35/40 - Loss: 0.2381 b_loss=0.2287 - T: 33.05s/epoch ,tempo_previto=1060.23 min0.005 nl5 QC256
35
T=10 Epoch: 36/40 - Loss: 0.2352 b_loss=0.2287 - T: 33.65s/epoch ,tempo_previto=1078.89 min0.005 nl5 QC256
36
T=10 Epoch: 37/40 - Loss: 0.2416 b_loss=0.2287 - T: 32.96s/epoch ,tempo_previto=1056.50 min0.005 nl5 QC256
37
T=10 Epoch: 38/40 - Loss: 0.2294 b_loss=0.2287 - T: 32.91s/epoch ,tempo_previto=1054.10 min0.005 nl5 QC256
38
T=10 Epoch

T=10 Epoch: 26/40 - Loss: 0.2544 b_loss=0.2544 - T: 33.01s/epoch ,tempo_previto=1152.13 min0.01 nl5 QC448
26
T=10 Epoch: 27/40 - Loss: 0.2525 b_loss=0.2525 - T: 32.95s/epoch ,tempo_previto=1149.36 min0.01 nl5 QC448
27
T=10 Epoch: 28/40 - Loss: 0.2537 b_loss=0.2525 - T: 33.02s/epoch ,tempo_previto=1151.32 min0.01 nl5 QC448
28
T=10 Epoch: 29/40 - Loss: 0.2483 b_loss=0.2483 - T: 32.93s/epoch ,tempo_previto=1147.57 min0.01 nl5 QC448
29
T=10 Epoch: 30/40 - Loss: 0.2579 b_loss=0.2483 - T: 33.05s/epoch ,tempo_previto=1151.34 min0.01 nl5 QC448
30
T=10 Epoch: 31/40 - Loss: 0.2592 b_loss=0.2483 - T: 33.17s/epoch ,tempo_previto=1154.71 min0.01 nl5 QC448
31
T=10 Epoch: 32/40 - Loss: 0.2531 b_loss=0.2483 - T: 33.16s/epoch ,tempo_previto=1153.87 min0.01 nl5 QC448
32
T=10 Epoch: 33/40 - Loss: 0.2582 b_loss=0.2483 - T: 32.98s/epoch ,tempo_previto=1147.31 min0.01 nl5 QC448
33
T=10 Epoch: 34/40 - Loss: 0.2479 b_loss=0.2479 - T: 33.04s/epoch ,tempo_previto=1148.61 min0.01 nl5 QC448
34
T=10 Epoch: 35/40 -

...

T=10 Epoch: 18/40 - Loss: 0.1281 b_loss=0.1266 - T: 134.29s/epoch ,tempo_previto=1481.66 min0.005 nl20 QC496
18
T=10 Epoch: 19/40 - Loss: 0.1285 b_loss=0.1266 - T: 130.78s/epoch ,tempo_previto=1440.72 min0.005 nl20 QC496
19
T=10 Epoch: 20/40 - Loss: 0.1210 b_loss=0.1210 - T: 132.00s/epoch ,tempo_previto=1451.95 min0.005 nl20 QC496
20
T=10 Epoch: 21/40 - Loss: 0.1286 b_loss=0.1210 - T: 134.05s/epoch ,tempo_previto=1472.27 min0.005 nl20 QC496
21
T=10 Epoch: 22/40 - Loss: 0.1229 b_loss=0.1210 - T: 132.73s/epoch ,tempo_previto=1455.58 min0.005 nl20 QC496
22
T=10 Epoch: 23/40 - Loss: 0.1214 b_loss=0.1210 - T: 133.63s/epoch ,tempo_previto=1463.22 min0.005 nl20 QC496
23
T=10 Epoch: 24/40 - Loss: 0.1231 b_loss=0.1210 - T: 132.15s/epoch ,tempo_previto=1444.79 min0.005 nl20 QC496
24
T=10 Epoch: 25/40 - Loss: 0.1206 b_loss=0.1206 - T: 132.54s/epoch ,tempo_previto=1446.88 min0.005 nl20 QC496
25
T=10 Epoch: 26/40 - Loss: 0.1195 b_loss=0.1195 - T: 132.06s/epoch ,tempo_previto=1439.47 min0.005 nl20 Q

T=10 Epoch: 12/40 - Loss: 0.1812 b_loss=0.1812 - T: 130.47s/epoch ,tempo_previto=1800.54 min0.01 nl20 QC510
12
T=10 Epoch: 13/40 - Loss: 0.1841 b_loss=0.1812 - T: 130.23s/epoch ,tempo_previto=1795.00 min0.01 nl20 QC510
13
T=10 Epoch: 14/40 - Loss: 0.1857 b_loss=0.1812 - T: 131.85s/epoch ,tempo_previto=1815.18 min0.01 nl20 QC510
14
T=10 Epoch: 15/40 - Loss: 0.1865 b_loss=0.1812 - T: 130.70s/epoch ,tempo_previto=1797.18 min0.01 nl20 QC510
15
T=10 Epoch: 16/40 - Loss: 0.1797 b_loss=0.1797 - T: 129.98s/epoch ,tempo_previto=1785.12 min0.01 nl20 QC510
16
T=10 Epoch: 17/40 - Loss: 0.1822 b_loss=0.1797 - T: 130.07s/epoch ,tempo_previto=1784.11 min0.01 nl20 QC510
17
T=10 Epoch: 18/40 - Loss: 0.1824 b_loss=0.1797 - T: 130.09s/epoch ,tempo_previto=1782.30 min0.01 nl20 QC510
18
T=10 Epoch: 19/40 - Loss: 0.1798 b_loss=0.1797 - T: 129.98s/epoch ,tempo_previto=1778.60 min0.01 nl20 QC510
19
T=10 Epoch: 20/40 - Loss: 0.1780 b_loss=0.1780 - T: 130.04s/epoch ,tempo_previto=1777.20 min0.01 nl20 QC510
20
T

T=10 Epoch: 6/40 - Loss: 0.1270 b_loss=0.1270 - T: 341.41s/epoch ,tempo_previto=3379.99 min0.05 nl50 QC0
6
T=10 Epoch: 7/40 - Loss: 0.1109 b_loss=0.1109 - T: 334.90s/epoch ,tempo_previto=3309.90 min0.05 nl50 QC0
7
T=10 Epoch: 8/40 - Loss: 0.1005 b_loss=0.1005 - T: 327.37s/epoch ,tempo_previto=3230.02 min0.05 nl50 QC0
8
T=10 Epoch: 9/40 - Loss: 0.0817 b_loss=0.0817 - T: 337.67s/epoch ,tempo_previto=3326.09 min0.05 nl50 QC0
9
T=10 Epoch: 10/40 - Loss: 0.0808 b_loss=0.0808 - T: 326.42s/epoch ,tempo_previto=3209.81 min0.05 nl50 QC0
10
T=10 Epoch: 11/40 - Loss: 0.0794 b_loss=0.0794 - T: 343.83s/epoch ,tempo_previto=3375.30 min0.05 nl50 QC0
11
T=10 Epoch: 12/40 - Loss: 0.0718 b_loss=0.0718 - T: 327.86s/epoch ,tempo_previto=3213.05 min0.05 nl50 QC0
12
T=10 Epoch: 13/40 - Loss: 0.0636 b_loss=0.0636 - T: 325.75s/epoch ,tempo_previto=3186.96 min0.05 nl50 QC0
13
T=10 Epoch: 14/40 - Loss: 0.0653 b_loss=0.0636 - T: 323.27s/epoch ,tempo_previto=3157.25 min0.05 nl50 QC0
14
T=10 Epoch: 15/40 - Loss: 0

T=10 Epoch: 2/40 - Loss: 0.1718 b_loss=0.1718 - T: 328.96s/epoch ,tempo_previto=1085.58 min0.005 nl50 QC0
2
T=10 Epoch: 3/40 - Loss: 0.1291 b_loss=0.1291 - T: 324.52s/epoch ,tempo_previto=1065.51 min0.005 nl50 QC0
3
T=10 Epoch: 4/40 - Loss: 0.0976 b_loss=0.0976 - T: 323.53s/epoch ,tempo_previto=1056.88 min0.005 nl50 QC0
4
T=10 Epoch: 5/40 - Loss: 0.0761 b_loss=0.0761 - T: 323.60s/epoch ,tempo_previto=1051.71 min0.005 nl50 QC0
5
T=10 Epoch: 6/40 - Loss: 0.0633 b_loss=0.0633 - T: 324.19s/epoch ,tempo_previto=1048.22 min0.005 nl50 QC0
6
T=10 Epoch: 7/40 - Loss: 0.0566 b_loss=0.0566 - T: 323.65s/epoch ,tempo_previto=1041.08 min0.005 nl50 QC0
7
T=10 Epoch: 8/40 - Loss: 0.0468 b_loss=0.0468 - T: 325.71s/epoch ,tempo_previto=1042.27 min0.005 nl50 QC0
8
T=10 Epoch: 9/40 - Loss: 0.0453 b_loss=0.0453 - T: 326.94s/epoch ,tempo_previto=1040.77 min0.005 nl50 QC0
9
T=10 Epoch: 10/40 - Loss: 0.0385 b_loss=0.0385 - T: 326.61s/epoch ,tempo_previto=1034.27 min0.005 nl50 QC0
10
T=10 Epoch: 11/40 - Loss: 

T=10 Epoch: 37/40 - Loss: 0.0479 b_loss=0.0466 - T: 334.75s/epoch ,tempo_previto=2917.94 min0.05 nl50 QC256
37
T=10 Epoch: 38/40 - Loss: 0.0499 b_loss=0.0466 - T: 325.22s/epoch ,tempo_previto=2829.45 min0.05 nl50 QC256
38
T=10 Epoch: 39/40 - Loss: 0.0510 b_loss=0.0466 - T: 325.20s/epoch ,tempo_previto=2823.82 min0.05 nl50 QC256
39
T=10 Epoch: 40/40 - Loss: 0.0460 b_loss=0.0460 - T: 326.50s/epoch ,tempo_previto=2829.70 min0.05 nl50 QC256
(11,)
0
T=10 Epoch: 1/40 - Loss: 0.2275 b_loss=0.2275 - T: 328.19s/epoch ,tempo_previto=1963.66 min0.01 nl50 QC256
1
T=10 Epoch: 2/40 - Loss: 0.1674 b_loss=0.1674 - T: 329.93s/epoch ,tempo_previto=1968.60 min0.01 nl50 QC256
2
T=10 Epoch: 3/40 - Loss: 0.1310 b_loss=0.1310 - T: 333.38s/epoch ,tempo_previto=1983.64 min0.01 nl50 QC256
3
T=10 Epoch: 4/40 - Loss: 0.0963 b_loss=0.0963 - T: 329.44s/epoch ,tempo_previto=1954.70 min0.01 nl50 QC256
4
T=10 Epoch: 5/40 - Loss: 0.0823 b_loss=0.0823 - T: 328.91s/epoch ,tempo_previto=1946.06 min0.01 nl50 QC256
5
T=10 E